In [1]:
from ultralytics import YOLO
from paddleocr import PaddleOCR
import cv2
import pandas as pd

OSError: [WinError 127] The specified procedure could not be found. Error loading "C:\Users\aarya\anaconda3\envs\tensorrt\lib\site-packages\paddle\..\nvidia\cudnn\bin\cudnn_engines_precompiled64_9.dll" or one of its dependencies.

In [1]:
# Load optimized models once
yolo_model = YOLO('runs/obb/train/weights/best.engine')
ocr = PaddleOCR(
    det_model_dir='det_infer_trt',
    rec_model_dir='rec_infer_trt', 
    use_gpu=True,
    use_tensorrt=True,
    gpu_mem=500
)

def process_image(image):
    predictions = yolo_model(image)[0]
    results = {'GTIN': '', 'SR_NO': '', 'LOT': '', 'EXP': ''}
    
    if predictions.obb:
        boxes = predictions.obb.xyxyxyxy.cpu().numpy()
        classes = predictions.obb.cls.cpu().numpy()
        
        for box, cls_id in zip(boxes, classes):
            box_int = box.astype(np.int32)
            x_min, y_min = np.min(box_int, axis=0)
            x_max, y_max = np.max(box_int, axis=0)
            
            roi = image[y_min:y_max, x_min:x_max]
            result = ocr.ocr(roi)
            
            if result[0]:
                text = result[0][0][1][0].strip()
                class_name = yolo_model.names[int(cls_id)]
                results[class_name] = clean_text(text, class_name)
                
    return results

# Use with webcam/video stream
def process_video():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = process_image(frame)
        # Add visualization/processing logic

[2024/12/19 12:06:44] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=True, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='det_infer_trt', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='rec_infer_trt', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='C:\\Users\\aarya\\anaconda3\\envs\\paddleocr\\lib\\site-pac

RuntimeError: (NotFound) TensorRT is needed, but TensorRT dynamic library is not found.
  Suggestions:
  1. Check if the TensorRT is installed correctly and its version is matched with paddlepaddle you installed.
  2. Configure environment variables as follows:
  - Linux: set LD_LIBRARY_PATH by `export LD_LIBRARY_PATH=...`
  - Windows: set PATH by `set PATH=XXX;
  [Hint: dso_handle should not be null.] (at ..\paddle\fluid\platform\dynload\tensorrt.cc:57)
